In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
File_address = '/home/haoyan/Desktop/ML-CFD/CASE/PHill/TBNN/h8-old1mesh/'
End = '.npy'
LES_address = File_address  + 'Anisotropy_Tensors_LES_NASA' + End
Tensors_address = File_address + 'Tensors_RANS' + End
Invariants_address = File_address + 'Invariants_RANS' + End
Anisotropy = np.load(LES_address)
Invariants = np.load(Invariants_address)
Two_Invariants = Invariants[:,:2]
General_Tensors = np.load(Tensors_address)
Five_Tensors = General_Tensors[:,:5]
Index = np.arange(Invariants.shape[0])
Input_shape = Invariants.shape[1]
Two_dim_anisotropy = np.zeros((93*83,4))
for i in range((93*83)):
    Two_dim_anisotropy[i][0] = Anisotropy[i][0]
    Two_dim_anisotropy[i][1] = Anisotropy[i][1]
    Two_dim_anisotropy[i][2] = Anisotropy[i][3]
    Two_dim_anisotropy[i][3] = Anisotropy[i][5]

X_train,X_val,Y_train,Y_val,Ind_train,Ind_val = train_test_split(Invariants,Anisotropy,Index,test_size=0.2, random_state=10, shuffle=True)
Basis_train = General_Tensors[Ind_train]
Basis_val = General_Tensors[Ind_val]
X_train = StandardScaler().fit_transform(X_train)
X_val = StandardScaler().fit_transform(X_val)##!Different from Example
print('Shape of Five_Tensors:',General_Tensors.shape)
print('Shape of Invariants:', Invariants.shape)
print('Shape of LES Data:', Anisotropy.shape)
print(' ')
print('Training Invariants:')
print(X_train.shape)
print('Training tensor basis:')
print(Basis_train.shape)
print('Training labels:')
print(Y_train.shape)
print(' ')
print('Validation Invariants:')
print(X_val.shape)
print('Validation tensor basis:')
print(Basis_val.shape)
print('Validation labels:')
print(Y_val.shape)
print(' ')

Shape of Five_Tensors: (7719, 10, 3, 3)
Shape of Invariants: (7719, 5)
Shape of LES Data: (7719, 6)
 
Training Invariants:
(6175, 5)
Training tensor basis:
(6175, 10, 3, 3)
Training labels:
(6175, 6)
 
Validation Invariants:
(1544, 5)
Validation tensor basis:
(1544, 10, 3, 3)
Validation labels:
(1544, 6)
 


In [18]:
import tensorflow as tf
from tensorflow import keras
keras.backend.clear_session()

input_layer = keras.layers.Input(shape=(Input_shape),name = 'Input_layer')
input_tensor_basis = keras.layers.Input(shape=(10,3,3),name = 'Tensor_Input_layer')

hidden1 = keras.layers.Dense(15,name='Hidden1', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "selu")(input_layer)
hidden2 = keras.layers.Dense(20,name='Hidden2', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "selu")(hidden1)
hidden3 = keras.layers.Dense(20,name='Hidden3', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "tanh")(hidden2)
hidden4 = keras.layers.Dense(20,name='Hidden4', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "selu")(hidden3)#
hidden5 = keras.layers.Dense(20,name='Hidden5', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-3), activation = "tanh")(hidden4)
gn = keras.layers.Dense(10,name='gn', kernel_initializer="lecun_normal",kernel_regularizer=tf.keras.regularizers.l2(1E-4), activation = "selu")(hidden5)

shaped = keras.layers.Reshape((10,1,1),name='Shape_for_dot_product')(gn)
merge = keras.layers.Dot(axes=1, name='Dot_product')([shaped,input_tensor_basis])

shaped_output = keras.layers.Reshape((9,1),name='Shaped_output')(merge)
trimmed_output1 = keras.layers.Lambda(lambda x : x[:,0])(shaped_output)
trimmed_output2 = keras.layers.Lambda(lambda x : x[:,1])(shaped_output)
trimmed_output3 = keras.layers.Lambda(lambda x : x[:,2])(shaped_output)
trimmed_output4 = keras.layers.Lambda(lambda x : x[:,4])(shaped_output)
trimmed_output5 = keras.layers.Lambda(lambda x : x[:,5])(shaped_output)
trimmed_output6 = keras.layers.Lambda(lambda x : x[:,8])(shaped_output)
merged_output = tf.keras.layers.Concatenate()([trimmed_output1,trimmed_output2,trimmed_output3,trimmed_output4,trimmed_output5,trimmed_output6])
#merged_output = tf.keras.layers.Concatenate()([trimmed_output1,trimmed_output2,trimmed_output4,trimmed_output6])
model=keras.Model(inputs=[input_layer,input_tensor_basis], outputs=[merged_output])

optimizer = tf.keras.optimizers.Nadam(learning_rate = 5E-4, clipnorm=1000)
model.compile(optimizer,loss='mse',metrics=['mae', 'mse'])
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_layer (InputLayer)       [(None, 5)]          0           []                               
                                                                                                  
 Hidden1 (Dense)                (None, 15)           90          ['Input_layer[0][0]']            
                                                                                                  
 Hidden2 (Dense)                (None, 20)           320         ['Hidden1[0][0]']                
                                                                                                  
 Hidden3 (Dense)                (None, 20)           420         ['Hidden2[0][0]']                
                                                                                              

In [19]:
reduce_lr =tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",factor=0.3,patience=10,verbose=0,mode="auto",min_delta=0.0001,cooldown=0,min_lr=0)
early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=0,patience=40,verbose=0,mode="auto",baseline=None,restore_best_weights=True)
history = model.fit([X_train,Basis_train],Y_train,batch_size=500,epochs=1000,validation_data = ([X_val,Basis_val],Y_val),verbose=1,callbacks=[early_stop,reduce_lr])

Epoch 1/1000
13/13 [==============================] - 1s 21ms/step - loss: 840.5317 - mae: 3.0168 - mse: 840.4333 - val_loss: 503.0289 - val_mae: 2.3519 - val_mse: 502.9306 - lr: 5.0000e-04
Epoch 2/1000
13/13 [==============================] - 0s 5ms/step - loss: 557.0440 - mae: 2.3691 - mse: 556.9457 - val_loss: 316.6815 - val_mae: 2.0179 - val_mse: 316.5832 - lr: 5.0000e-04
Epoch 3/1000
13/13 [==============================] - 0s 5ms/step - loss: 352.7296 - mae: 1.9241 - mse: 352.6314 - val_loss: 121.6965 - val_mae: 1.5463 - val_mse: 121.5983 - lr: 5.0000e-04
Epoch 4/1000
13/13 [==============================] - 0s 5ms/step - loss: 196.4005 - mae: 1.5075 - mse: 196.3023 - val_loss: 81.5802 - val_mae: 1.3656 - val_mse: 81.4820 - lr: 5.0000e-04
Epoch 5/1000
13/13 [==============================] - 0s 5ms/step - loss: 129.6254 - mae: 1.2430 - mse: 129.5272 - val_loss: 36.3126 - val_mae: 1.1207 - val_mse: 36.2144 - lr: 5.0000e-04
Epoch 6/1000
13/13 [==============================] - 0s 5

13/13 [==============================] - 0s 5ms/step - loss: 0.9628 - mae: 0.2932 - mse: 0.8651 - val_loss: 13.8077 - val_mae: 0.3727 - val_mse: 13.7099 - lr: 1.5000e-04
Epoch 46/1000
13/13 [==============================] - 0s 5ms/step - loss: 0.9078 - mae: 0.2905 - mse: 0.8101 - val_loss: 13.6883 - val_mae: 0.3684 - val_mse: 13.5905 - lr: 1.5000e-04
Epoch 47/1000
13/13 [==============================] - 0s 5ms/step - loss: 0.8444 - mae: 0.2874 - mse: 0.7466 - val_loss: 14.6095 - val_mae: 0.3834 - val_mse: 14.5117 - lr: 1.5000e-04
Epoch 48/1000
13/13 [==============================] - 0s 5ms/step - loss: 0.8106 - mae: 0.2879 - mse: 0.7128 - val_loss: 13.9469 - val_mae: 0.3657 - val_mse: 13.8491 - lr: 1.5000e-04
Epoch 49/1000
13/13 [==============================] - 0s 5ms/step - loss: 0.7577 - mae: 0.2833 - mse: 0.6600 - val_loss: 14.9167 - val_mae: 0.3856 - val_mse: 14.8190 - lr: 1.5000e-04
Epoch 50/1000
13/13 [==============================] - 0s 5ms/step - loss: 0.7598 - mae: 0.285

In [20]:
model.evaluate([X_val,Basis_val],Y_val)

49/49 [==============================] - 0s 2ms/step - loss: 3.3656 - mae: 0.5982 - mse: 3.2676


[3.3655614852905273, 0.5981546640396118, 3.2675540447235107]

In [22]:
w = model.layers[6].weights[1].numpy()
p_b = np.zeros((93*83,3,3))
for i in range(93*83):
    for j in range(10):
        p_b[i] += w[j] * General_Tensors[i][j] 
np.save("/home/haoyan/Desktop/ML-CFD/CASE/PHill/TBNN/p_b_TBNN.npy",p_b)

In [21]:
model.layers[6].weights[1].numpy()

array([-0.02016694,  0.00895048,  0.01445335, -0.00869193, -0.01291759,
        0.02554571, -0.00631413, -0.00340419, -0.00435386,  0.01818621],
      dtype=float32)